In [1]:
import os
import sys
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from kneed import KneeLocator
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.metrics import silhouette_score
from sklearn.cluster import AffinityPropagation
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.cluster import SpectralClustering
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.cluster import OPTICS
from sklearn.cluster import Birch
import matplotlib


In [2]:
def blockPrint():
    sys.__stdout__ = sys.stdout
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


In [3]:
def min_max_normalization(df_col):
    data_min, data_max = numpy.min(df_col),numpy.max(df_col)
    normalized = [(item - data_min)/(data_max - data_min) for item in df_col]
    return normalized

In [4]:
def my_HierarchicalClustering(df, linkage, k=None):
    blockPrint()
    #d_array = numpy.array(data_list)
    # Find K
    if not k:
        #km_kwargs = {"init":"random", "n_init":10,"max_iter": 3000000, "random_state" : 50}
        silhouette = []
        for k in range(2, 10):
            hc = AgglomerativeClustering(n_clusters=k, linkage = linkage)
            hc.fit(df)
            silhouette.append(silhouette_score(df,hc.labels_))

        plt.style.use("fivethirtyeight")
        plt.plot(range(2, 10), silhouette)
        plt.xticks(range(2, 10))
        plt.xlabel("Number of Clusters")
        plt.ylabel("silhouette coefficient")
        plt.show()  
        #Find K
        kl = KneeLocator(range(2, 10), silhouette, curve="convex", direction="decreasing")
        k = kl.elbow
        print(f"elbow point = {k}")

    d_hc = AgglomerativeClustering(n_clusters=k, linkage = linkage)
    d_hc.fit(df)
    labels = list(d_hc.labels_)
    enablePrint()
    
    return labels

In [5]:
def my_SpectralClustering(df, k=None):
    #d_array = numpy.array(data_list)
    # Find K
    blockPrint()
    if not k:
        #km_kwargs = {"init":"random", "n_init":10,"max_iter": 3000000, "random_state" : 50}
        silhouette = []
        for k in range(2, 10):
            sc = SpectralClustering(n_clusters=k, assign_labels='cluster_qr')
            sc.fit(df)
            silhouette.append(silhouette_score(df,sc.labels_))

        plt.style.use("fivethirtyeight")
        plt.plot(range(2, 10), silhouette)
        plt.xticks(range(2, 10))
        plt.xlabel("Number of Clusters")
        plt.ylabel("silhouette coefficient")
        plt.show()  
        #Find K
        kl = KneeLocator(range(2, 10), silhouette, curve="convex", direction="decreasing")
        k = kl.elbow
        print(f"elbow point = {k}")

    d_sc = SpectralClustering(n_clusters=k, assign_labels='discretize')
    d_sc.fit(df)
    labels = list(d_sc.labels_)
    enablePrint()
    
    return labels

In [6]:
def my_kmeans(df, k=None):
    #d_array = numpy.array(data_list)
    # Find K
    blockPrint()
    if not k:
        km_kwargs = {"init":"random", "n_init":10,"max_iter": 3000000, "random_state" : 50}
        cost = []

        for k in range(1, 11):
            km = KMeans(n_clusters=k, **km_kwargs)
            km.fit(df)
            cost.append(km.inertia_)

        plt.style.use("fivethirtyeight")
        plt.plot(range(1, 11), cost)
        plt.xticks(range(1, 11))
        plt.xlabel("Number of Clusters")
        plt.ylabel("Cost")
        plt.show()  
        #Find K
        kl = KneeLocator(range(1, 11), cost, curve="convex", direction="decreasing")
        k = kl.elbow
        print(f"elbow point = {k}")

    d_km = KMeans( n_clusters=k, init="random", n_init=10, verbose=1, random_state = 100)
    d_km.fit(df)
    labels = list(d_km.labels_)
    enablePrint()
    
    return labels

In [ ]:
os.getcwd()

In [8]:
basedir = os.path.join("..","data")
developer_data = pd.read_csv(os.path.join(basedir, "contributor_features.csv"), index_col=0)

In [ ]:
cluster_data1 = developer_data[["worktime", "code contribution density", "languages", "total commits"]]

for fea in ["code contribution density", "languages", "total commits"]:
    q1,q2,q3 = developer_data[fea].quantile([.5,.75,.9])
    cluster_data1.loc[developer_data[fea] < q1, fea] = 0
    cluster_data1.loc[(developer_data[fea] >= q1) & (developer_data[fea] < q2), fea] = 1
    cluster_data1.loc[(developer_data[fea] >= q2) & (developer_data[fea] < q3), fea] = 2
    cluster_data1.loc[(developer_data[fea] >= q3) , fea] = 3

cluster_data1.loc[ (developer_data["worktime"]>=1)&(developer_data["worktime"]<=7),"worktime"] = 3
cluster_data1.loc[ (developer_data["worktime"]>=8)&(developer_data["worktime"]<=18),"worktime"] = 0
cluster_data1.loc[ (developer_data["worktime"]>=19)&(developer_data["worktime"]<=23),"worktime"] = 3
cluster_data1.loc[ developer_data["worktime"]==0,"worktime"] = 3

cluster_data1["code contribution density"] = cluster_data1["code contribution density"].astype(int)
cluster_result1 = developer_data[["name", "project"]]

### kmeans

In [ ]:
km_labels = my_kmeans(cluster_data1)
cluster_result1["kmeans"] = km_labels

In [ ]:
cluster_result1["kmeans"].value_counts()

In [ ]:
silhouette_score(cluster_data1,km_labels)

### affinity propogation

In [ ]:
for i in range(100):
    af = AffinityPropagation(preference=i*(-1), random_state=0).fit(cluster_data1)
    cluster_centers_indices = af.cluster_centers_indices_
    af_labels = af.labels_

    n_clusters_ = len(cluster_centers_indices)
    if n_clusters_<= 1:
        print(i*(-1), "Estimated number of clusters: %d" % n_clusters_)
    else:
        print(i*(-1), "Estimated number of clusters: %d" % n_clusters_, silhouette_score(cluster_data1,af_labels))

In [ ]:
af = AffinityPropagation(preference=-6, random_state=0).fit(cluster_data1)
cluster_centers_indices = af.cluster_centers_indices_
af_labels = af.labels_

n_clusters_ = len(cluster_centers_indices)
cluster_result1["af"] = af_labels
print("Estimated number of clusters: %d" % n_clusters_,silhouette_score(cluster_data1,af_labels))

In [ ]:
print(davies_bouldin_score(cluster_data1,cluster_result1['af']),calinski_harabasz_score(cluster_data1,cluster_result1['af']))

### means shift

In [ ]:
for i in [x*0.1 for x in range(1,11)]:
    bandwidth = estimate_bandwidth(cluster_data1, quantile=i, n_samples=500)
    
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(cluster_data1)
    ms_labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    
    labels_unique = np.unique(ms_labels)
    n_clusters_ = len(labels_unique)
    
    if n_clusters_<= 1:
        print(i, "Estimated number of clusters: %d" % n_clusters_)
    else:
        print(i, "Estimated number of clusters: %d" % n_clusters_, silhouette_score(cluster_data1,ms_labels))

In [ ]:
for i in [1000,800, 500, 100, 90, 80, 60, 50, 40, 30, 20]:
    bandwidth = estimate_bandwidth(cluster_data1, quantile=0.3, n_samples=i)
    
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(cluster_data1)
    ms_labels = ms.labels_
    cluster_centers = ms.cluster_centers_
    
    labels_unique = np.unique(ms_labels)
    n_clusters_ = len(labels_unique)
    
    if n_clusters_<= 1:
        print(i, "Estimated number of clusters: %d" % n_clusters_)
    else:
        print(i, "Estimated number of clusters: %d" % n_clusters_, silhouette_score(cluster_data1,ms_labels))

In [ ]:
bandwidth = estimate_bandwidth(cluster_data1, quantile=0.3, n_samples=100)
ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(cluster_data1)
ms_labels = ms.labels_
cluster_result1['ms'] = ms_labels
silhouette_score(cluster_data1,ms_labels)

### spectral clustering

In [ ]:
sc_labels = my_SpectralClustering(cluster_data1)

In [ ]:
sc = SpectralClustering(n_clusters=4,assign_labels='cluster_qr',random_state=0).fit(cluster_data1)
sc_labels = sc.labels_
cluster_result1["sc"] = sc_labels
silhouette_score(cluster_data1,sc_labels)

### hierarchical clustering

In [ ]:
hc_labels = my_HierarchicalClustering(cluster_data1,"ward")

In [ ]:
hc = AgglomerativeClustering(n_clusters=3, linkage="ward").fit(cluster_data1)
hc_labels = hc.labels_
labels_unique = np.unique(hc_labels)
n_clusters_ = len(labels_unique)

cluster_result1["hc"] = hc_labels
print("number of estimated clusters : %d" % n_clusters_, silhouette_score(cluster_data1,hc_labels))

### dbscan

In [ ]:
for i in [0.1*x for x in range(1,10)]:
    dbscan = DBSCAN(eps=i,min_samples=50).fit(cluster_data1)
    db_labels = dbscan.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(db_labels)) - (1 if -1 in db_labels else 0)
    n_noise_ = list(db_labels).count(-1)

    print(i,"Estimated number of clusters: %d" % n_clusters_)
    print(i,"Estimated number of noise points: %d" % n_noise_)
    print(silhouette_score(cluster_data1,db_labels))


In [ ]:
for n in range(1,500,10):   
    dbscan = DBSCAN(eps=0.1,min_samples=n).fit(cluster_data1)
    db_labels = dbscan.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(db_labels)) - (1 if -1 in db_labels else 0)
    n_noise_ = list(db_labels).count(-1)
    #if n_clusters_ < 10:
    print(n)
    print(n,"Estimated number of clusters: %d" % n_clusters_)
    print(n, "Estimated number of noise points: %d" % n_noise_)
    print(silhouette_score(cluster_data1,db_labels))

In [ ]:
dbscan = DBSCAN(eps=0.1,min_samples=171).fit(cluster_data1)
db_labels = dbscan.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(db_labels)) - (1 if -1 in db_labels else 0)
n_noise_ = list(db_labels).count(-1)
#if n_clusters_ < 10:
print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)
print(silhouette_score(cluster_data1,db_labels))
cluster_result1["dbscan"] = db_labels

### optics

In [ ]:
for i in [5000, 4000, 3000, 2000, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 150, 100, 90, 80, 60, 50, 40, 30, 20, 10,9,8,7,6,5,4,3,2]:
    optics = OPTICS(eps=0.1,min_samples=i).fit(cluster_data1)
    optics_labels = optics.labels_
    
    labels_unique = np.unique(optics_labels)
    n_clusters_ = len(labels_unique)
    if n_clusters_ > 1:
        print(i,"number of estimated clusters : %d" % n_clusters_,silhouette_score(cluster_data1,optics_labels))
    else:
        print(i,"number of estimated clusters : %d" % n_clusters_)

In [ ]:
for i in [x*0.1 for x in range(1,11)]:
    optics = OPTICS(eps=i,min_samples=4000).fit(cluster_data1)
    optics_labels = optics.labels_
    
    labels_unique = np.unique(optics_labels)
    n_clusters_ = len(labels_unique)
    
    if n_clusters_ > 1:
        print(i,"number of estimated clusters : %d" % n_clusters_,silhouette_score(cluster_data1,optics_labels))
    else:
        print(i,"number of estimated clusters : %d" % n_clusters_)

In [ ]:
optics = OPTICS(eps=0.1,min_samples=4000).fit(cluster_data1)
optics_labels = optics.labels_
labels_unique = np.unique(optics_labels)
n_clusters_ = len(labels_unique)
cluster_result1["optics"] = optics_labels
silhouette_score(cluster_data1,optics_labels)

### BIRCH

In [ ]:
#for i in [5000, 4000, 3000, 2000, 1000, 900, 800, 700, 600, 500, 400, 300, 200, 150, 100, 90, 80, 60, 50, 40, 30, 20, 10,9,8,7,6,5,4,3,2]:
for k in range(2,11):
    brc = Birch(threshold=1.0, branching_factor=500, n_clusters=k).fit(np.ascontiguousarray(cluster_data1))
    brc_labels = brc.labels_
    
    labels_unique = np.unique(brc_labels)
    n_clusters_ = len(labels_unique)
    
    if n_clusters_ > 1:
        print(k,"number of estimated clusters : %d" % n_clusters_,silhouette_score(cluster_data1,brc_labels))
    else:
        print(k,"number of estimated clusters : %d" % n_clusters_)

In [ ]:
brc = Birch(threshold=1.0, branching_factor=500, n_clusters=4).fit(np.ascontiguousarray(cluster_data1))
brc_labels = brc.labels_

labels_unique = np.unique(brc_labels)
n_clusters_ = len(labels_unique)

print(i, "number of estimated clusters : %d" % n_clusters_)
cluster_result1["birch"] = brc_labels

In [ ]:
cluster_result1

In [ ]:
for alg in ['kmeans',	'ms',	'sc',	'hc',	'dbscan',	'optics',	'birch']:
    print(alg, len(cluster_result1[alg].value_counts()), 
          silhouette_score(cluster_data1,cluster_result1[alg]))

## Clustering Results - Profile

In [ ]:
cluster_result1.loc[cluster_result1['sc']==0, 'profile'] = 'ca'
cluster_result1.loc[cluster_result1['sc']==1, 'profile'] = 'cw'
cluster_result1.loc[cluster_result1['sc']==2, 'profile'] = 'pa'
cluster_result1.loc[cluster_result1['sc']==3, 'profile'] = 'pw'

In [10]:
developer_data['profile'] = cluster_result1['profile']

In [ ]:
cluster_result1['profile'].value_counts()

## Add a column 'profile' to contributor_features.csv

In [11]:
developer_data.to_csv(os.path.join(basedir, "contributor_features.csv"))